In [1]:
import time
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

# Foursquare数据集分析

In [2]:
#实验室路径
#原始数据路径
#种类
file_category = 'D:\\data\\foursquare\\categories.txt'
#用户
    #LA
file_LAUsers = 'D:\\data\\foursquare\\Users\\LA\\LA-Users.txt'
    #NYC
file_NYCUsers = 'D:\\data\\foursquare\\Users\\NYC\\NYC-Users.txt'
#POI
    #LA
file_LAPOI = 'D:\\data\\foursquare\\Venues\\LA\\LA_POIs.csv'
    #NYC
file_NYCPOI = 'D:\\data\\foursquare\\Venues\\NYC\\NYC_POIs.csv'
#Check-ins
    #LA
file_LACheck = 'D:\\data\\foursquare\\Tips\\LA\\LA-Tips.txt'
    #NYC
file_NYCCheck = 'D:\\data\\foursquare\\Tips\\NYC\\NYC-Tips.txt'

## 1.种类分析

In [4]:
with open(file_category,'r',errors='ignore') as f:
    data = json.load(f)

In [5]:
cat = data['response']['categories']
cat[1]

{'categories': [{'categories': [{'icon': 'https://foursquare.com/img/categories/education/default.png',
     'id': '4bf58dd8d48988d199941735',
     'name': 'College Arts Building',
     'pluralName': 'College Arts Buildings'},
    {'icon': 'https://foursquare.com/img/categories/education/default.png',
     'id': '4bf58dd8d48988d19a941735',
     'name': 'College Communications Building',
     'pluralName': 'College Communications Buildings'},
    {'icon': 'https://foursquare.com/img/categories/education/default.png',
     'id': '4bf58dd8d48988d19e941735',
     'name': 'College Engineering Building',
     'pluralName': 'College Engineering Buildings'},
    {'icon': 'https://foursquare.com/img/categories/education/default.png',
     'id': '4bf58dd8d48988d19d941735',
     'name': 'College History Building',
     'pluralName': 'College History Buildings'},
    {'icon': 'https://foursquare.com/img/categories/education/default.png',
     'id': '4bf58dd8d48988d19c941735',
     'name': 'College

In [6]:
#统计不同种类的数目
l2 = [];l3 = []
for level2 in cat:
    l2.append(len(level2['categories']))
    nums = 0
    for level3 in level2['categories']:
        nums += len(level3['categories'])
    l3.append(nums)
print(l2)
print("第二级种类总数：-----%d-----" % sum(l2))
print(l3)
print("第三级种类总数：-----%d-----" % sum(l3))

[17, 23, 78, 28, 15, 20, 45, 15]
第二级种类总数：-----241-----
[22, 14, 1, 4, 28, 4, 18, 12]
第三级种类总数：-----103-----


In [7]:
name = pd.DataFrame(cat,columns = ['name'])
pd.DataFrame({'category':name['name'],'n2':l2,'n3':l3},columns = ['category','n2','n3'],index = range(0,8))

,category,n2,n3
0,Arts & Entertainment,17,22
1,College & University,23,14
2,Food,78,1
3,Great Outdoors,28,4
4,"Home, Work, Other",15,28
5,Nightlife Spot,20,4
6,Shop,45,18
7,Travel Spot,15,12


In [42]:
#种类整理
category = []   #用于存储所有的种类id和name，分层次,三层
categories = {} #用于存储所有的种类id和name，不分层次,总共352类
for i,level1 in enumerate(cat):  #第一级种类
    dict1 = {'name':level1['name']}
    categories[i] = level1['name']
    category1 = []
    for level2 in level1['categories']:
        dict2 = {'id':level2['id'],'name':level2['name']}
        categories[level2['id']] = level2['name']
        if len(level2['categories']) != 0:
            category2 = []
            for level3 in level2['categories']:
                dict3 = {'id':level3['id'],'name':level3['name']}
                categories[level3['id']] = level3['name']
                category2.append(dict3)
            dict2['categories'] = category2
        else:
            dict2['categories'] = []
        category1.append(dict2)
    dict1['categories'] = category1
    #else:
        #continue
    category.append(dict1)
        

In [43]:
n2 = [];n3 = []
for level2 in category:
    n2.append(len(level2['categories']))
    num3 = 0
    for level3 in level2['categories']:
        num3 += len(level3['categories'])
    n3.append(num3)
print("第一级种类总数：-----8------")
print(n2)
print("第二级种类总数：-----%d-----" % sum(n2))
print(n3)
print("第三级种类总数：-----%d-----" % sum(n3))
print("Foursquare种类的总数为：-----%d-----" % len(categories))

第一级种类总数：-----8------
[17, 23, 78, 28, 15, 20, 45, 15]
第二级种类总数：-----241-----
[22, 14, 1, 4, 28, 4, 18, 12]
第三级种类总数：-----103-----
Foursquare种类的总数为：-----352-----


In [44]:
#pd.DataFrame({'id':categories.keys(),'name':categories.values()},index = range(len(categories)),columns = ['id','name'])
ser = pd.Series(categories)
ser.to_csv("D:\\data\\foursquare\\csv\\category.csv") 

# 2.User 分析

## 2.1 LA Users

### 原始数据LAusers整理

In [91]:
user_LA = pd.read_table(file_LAUsers, sep = '\t',names = ['id','first name','last name','http','gender','home city'] )

In [92]:
user_LA = user_LA[['id','gender','home city']]
user_LA.head()

,id,gender,home city
0,528470,male,"London, United Kingdom"
1,525576,male,"Arlington, TX"
2,15420,female,"San Francisco, CA"
3,387,male,"New York, NY"
4,90398,female,"London, UK"


In [93]:
user = user_LA[['id','home city']] #user中只有id和home_city
user.index = user['id']  #将id作为索引便于后续merge
user = user['home city']
user = user[user.notnull()] #筛选home city非空的样本
user = pd.DataFrame(user.map(lambda x: x.split(',')))  #利用map函数将每个样本的home city字符串的形式分割成【city，state】的字符列表形式
user_LA = pd.merge(user_LA,user,left_on = 'id',right_index = True,how = 'left') 
del user_LA['home city_x']
user_LA.columns = ['id','gender','home']
user_LA['city'] = user_LA['home'] 
user_LA['state'] = user_LA['home']
user_LA.head()

,id,gender,home,city,state
0,528470,male,"[London, United Kingdom]","[London, United Kingdom]","[London, United Kingdom]"
1,525576,male,"[Arlington, TX]","[Arlington, TX]","[Arlington, TX]"
2,15420,female,"[San Francisco, CA]","[San Francisco, CA]","[San Francisco, CA]"
3,387,male,"[New York, NY]","[New York, NY]","[New York, NY]"
4,90398,female,"[London, UK]","[London, UK]","[London, UK]"


In [98]:
#f1函数用于提取city字符
def f1(x):
    if len(x) == 2:
        return x[0]
    elif len(x) == 1 and (x[0] in city_name): ##city_name为LA_user文件中所有城市名称的数组
        return x[0]
    else:
        return np.nan
#f2函数用于提取state字符   
def f2(x):
    if len(x) == 2:
        return x[1]
    elif len(x) == 1 and (x[0] in state_name):
        return x[0]
    else:
        return np.nan

In [95]:
home = user_LA[['id','home','city','state']]
home.index = home['id']
del home['id']
home = home[pd.notnull(home['home'])]

city = home['city']  #用于提取保存用户city项
state = home['city'] #用于提取保存用户state项
city = pd.DataFrame(city.map(f1))
state = pd.DataFrame(state.map(f2))

user_LA = pd.merge(user_LA[['id','gender','home']],city,left_on = 'id',right_index = True,how = 'left')
user_LA = pd.merge(user_LA,state,left_on = 'id',right_index = True,how = 'left')
user_LA.columns = ['id','gender','home','city','state']
user_LA.to_csv('D:\\data\\foursquare\\csv\\users_LA.csv')
user_LA.head()

,id,gender,home,city,state
0,528470,male,"[London, United Kingdom]",London,United Kingdom
1,525576,male,"[Arlington, TX]",Arlington,TX
2,15420,female,"[San Francisco, CA]",San Francisco,CA
3,387,male,"[New York, NY]",New York,NY
4,90398,female,"[London, UK]",London,UK


In [14]:
print("LA的用户数:------%d-----" % len(user_LA))

LA的用户数:------501940-----


### 整理数据加载分析

In [101]:
file_users_LA = 'D:\\data\\foursquare\\csv\\users_LA.csv'
users_LA = pd.read_csv(file_users_LA, sep = ',', encoding = 'latin-1',names = ['id','gender','home','city','state'])
users_LA.head()

D:\Python\Install\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,gender,home,city,state
NaN,id,gender,home,city,state
0.0,528470,male,"['London', ' United Kingdom']",London,United Kingdom
1.0,525576,male,"['Arlington', ' TX']",Arlington,TX
2.0,15420,female,"['San Francisco', ' CA']",San Francisco,CA
3.0,387,male,"['New York', ' NY']",New York,NY


In [102]:
ser_city = users_LA['city']
ser_state = users_LA['state']
ser_home = users_LA['home']
city_name = ser_city.unique()
state_name = ser_state.unique()

## NYC Users

In [105]:
user_NYC = pd.read_table(file_NYCUsers, sep = '\t',names = ['id','first name','last name','http','gender','home city'] )

In [106]:
user_NYC = user_NYC[['id','gender','home city']]
user_NYC.head()

,id,gender,home city
0,1554,female,"Venice, CA"
1,122155,male,"New Haven, CT"
2,855,male,"Portland, OR"
3,35490,female,"New York, NY"
4,304667,male,"Houston, TX"


In [107]:
user = user_NYC[['id','home city']] #user中只有id和home_city
user.index = user['id']  #将id作为索引便于后续merge
user = user['home city']
user = user[user.notnull()] #筛选home city非空的样本
user = pd.DataFrame(user.map(lambda x: x.split(',')))  #利用map函数将每个样本的home city字符串的形式分割成【city，state】的字符列表形式
user_NYC = pd.merge(user_NYC,user,left_on = 'id',right_index = True,how = 'left') 
del user_NYC['home city_x']
user_NYC.columns = ['id','gender','home']
user_NYC['city'] = user_NYC['home'] 
user_NYC['state'] = user_NYC['home']
user_NYC.head()

,id,gender,home,city,state
0,1554,female,"[Venice, CA]","[Venice, CA]","[Venice, CA]"
1,122155,male,"[New Haven, CT]","[New Haven, CT]","[New Haven, CT]"
2,855,male,"[Portland, OR]","[Portland, OR]","[Portland, OR]"
3,35490,female,"[New York, NY]","[New York, NY]","[New York, NY]"
4,304667,male,"[Houston, TX]","[Houston, TX]","[Houston, TX]"


In [108]:
#f1函数用于提取city字符
def f1(x):
    if len(x) == 2:
        return x[0]
    elif len(x) == 1 and (x[0] in city_name): ##city_name为LA_user文件中所有城市名称的数组
        return x[0]
    else:
        return np.nan
#f2函数用于提取state字符   
def f2(x):
    if len(x) == 2:
        return x[1]
    elif len(x) == 1 and (x[0] in state_name):
        return x[0]
    else:
        return np.nan

In [109]:
home = user_NYC[['id','home','city','state']]
home.index = home['id']
del home['id']
home = home[pd.notnull(home['home'])]

city = home['city']  #用于提取保存用户city项
state = home['city'] #用于提取保存用户state项
city = pd.DataFrame(city.map(f1))
state = pd.DataFrame(state.map(f2))

user_NYC = pd.merge(user_NYC[['id','gender','home']],city,left_on = 'id',right_index = True,how = 'left')
user_NYC = pd.merge(user_NYC,state,left_on = 'id',right_index = True,how = 'left')
user_NYC.columns = ['id','gender','home','city','state']
user_NYC.to_csv('D:\\data\\foursquare\\csv\\users_NYC.csv')
user_NYC.head()

,id,gender,home,city,state
0,1554,female,"[Venice, CA]",Venice,CA
1,122155,male,"[New Haven, CT]",New Haven,CT
2,855,male,"[Portland, OR]",Portland,OR
3,35490,female,"[New York, NY]",New York,NY
4,304667,male,"[Houston, TX]",Houston,TX


In [20]:
print("NYC的用户数:------%d-----" % len(user_NYC))

NYC的用户数:------717382-----


### 整理后数据加载

In [103]:
file_users_NYC = 'D:\\data\\foursquare\\csv\\users_NYC.csv'
users_NYC = pd.read_csv(file_users_NYC, sep = ',', encoding = 'latin-1',names = ['id','gender','home','city','state'])
users_NYC.head()

D:\Python\Install\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,gender,home,city,state
0,id,gender,home,city,state
1,1554,female,"['Venice', ' CA']",Venice,CA
2,122155,male,"['New Haven', ' CT']",New Haven,CT
3,855,male,"['Portland', ' OR']",Portland,OR
4,35490,female,"['New York', ' NY']",New York,NY


# 3. POI 分析

## LA POI

In [15]:
POI_LA = pd.read_csv(file_LAPOI,encoding = 'latin-1')

D:\Python\Install\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (5,6,8,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
# 'id','name','latitude','longtitude','address','city','state','checkin','checked user','current user','todo','no','category','category id 0','category id 1'

In [19]:
POI_LA

,4b382d99f964a520854c25e3,33.7482354,-118.4071307,Rancho Palos Verdes,CA,29,18,0,0.1,0.2,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,4be5acd52468c928cf5a0043,33.729714,-118.349261,Rancho Palos Verdes,CA,94,67,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4bf98d5e5317a593ef7a017f,33.7992814,-118.3510283,Torrance,CA,30,28,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4b8aed83f964a520a68932e3,33.8198319,-118.3883947,Redondo Beach,California,323,199,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4b91842ff964a5204ec233e3,33.74482527,-118.411417,Rancho Palos Verdes,CA,203,139,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4c11b66bd41e76b0bbfd310d,33.8286692,-118.2773108,Carson,California,37,23,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4a32e7b9f964a520bc9a1fe3,33.936578,-118.369507,Inglewood,CA,30,20,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4c250f90db519521291d2c3a,33.873156,-118.356795,Lawndale,CA,17,9,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,45057c3bf964a52001391fe3,33.842623,-118.2883841,Torrance,CA,792,548,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4c4e276567929c747ab6709b,33.774793,-118.417702,Palos Verdes Estates,Ca,32,19,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4ca7b00dd971b1f749f10ce1,33.7882828,-118.325411,Lomita,CA,132,2,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
print('LA的POI数目：------%d------' % len(POI_LA))

LA的POI数目：------215614------


In [21]:
file1 = r'E:\data\Foursquare\Microsoft_check-in data\Venues\LA\LA-Venues.txt'
file2 = r'D:\software\Github\Recommender-System\RS-LBSN\dataset\Foursquare\LA-2.txt'
f1 = open(file1,'r',errors = 'ignore')
f2 = open(file2,'w')
lines = f1.readlines()
for line in lines[0:144421]:
    cat = []
    lst = line.strip().split('\t')
    if len(lst) < 14:
        f2.write('\t'.join(lst))
        f2.write('\n')
    else:
        for e in lst[13:len(lst)]:
            if e != np.nan:
                cat.append(e[1:len(e)-1])
        lst.insert(13,','.join(cat))
        f2.write('\t'.join(lst[0:14]))
        f2.write('\n')
for line in lines[144421:]:
    cat = []
    lst = line.strip().split('\t')
    lst.insert(9,'0')
    if len(lst) < 14:
        f2.write('\t'.join(lst))
        f2.write('\n')
    else:
        for e in lst[13:len(lst)]:
            if e != np.nan:
                cat.append(e[1:len(e)-1])
        lst.insert(13,','.join(cat))
        f2.write('\t'.join(lst[0:14]))
        f2.write('\n')
f1.close()
f2.close()

In [16]:
file1 = r'E:\data\Foursquare\Microsoft_check-in data\Venues\LA\LA-Venues.txt'
file2 = r'D:\software\Github\Recommender-System\RS-LBSN\dataset\Foursquare\LA-2.txt'
f1 = open(file1,'r',errors = 'ignore')
f2 = open(file2,'w')
lines = f1.readlines()
for line in lines[144420:144426]:
    print(line)
f1.close()

"4ca2eda75e3076b0e2ceef0c"	"The Darling Nikki"	34.3779062117386	-118.573808670044	"Middle Earth"	"Los Angeles"	"CA"	29	7	0	0	0	2	"4bf58dd8d48988d103941735"	"4bf58dd8d48988d11f941735"

"4bf5d8e19abec9b622de24e8"	"Palos Verdes High School"	33.7797381666667	-118.41887797	"800 Cloyden Road"	"Palos Verdes"	"CA"	166	61	0	0	1	"4bf58dd8d48988d13d941735"

"4b0d9174f964a520384b23e3"	"Lunada Bay Market"	33.7754999275293	-118.418068885803	"2201 Palos Verdes Dr W"	"Palos Verdes Estates"	"CA"	190	42	0	0	1	"4bf58dd8d48988d118951735"

"4b3f9af2f964a52028aa25e3"	"Lunada Bay"	33.7714421198399	-118.42248916626	"2300 Paseo del Mar"	"Palos Verdes Estates"	"CA"	251	77	1	0	2	"4bf58dd8d48988d1e3941735"	"4bf58dd8d48988d165941735"

"4d7ff7638edaa143dca0e539"	"OnStratus - Palos Verdes West"	33.759922	-118.413987	null	null	null	95	2	0	0	1	"4bf58dd8d48988d125941735"

"4bddc93e0ee3a5938ba32eb0"	"Cliffs At Point Vicente"	33.7531263	-118.4122549	"Calle Entradero"	"Rancho Palos Verdes"	"California"	50	32	0	0	0



## NYC POI

In [13]:
POI_NYC = pd.read_csv(file_NYCPOI,encoding = 'latin-1')

D:\Python\Install\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (6,19,20,21,22,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
POI_NYC

,4c94b5f082b56dcbae61d1aa,40.403035,-74.35464,Old Bridge,NJ,9,3,0,0.1,0.2,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,4b7c9374f964a520e99b2fe3,38.55618663,-90.39645195,null,null,133,108,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4bd8d2132e6f0f4748c80808,40.567686,-74.548777,Bound Brook,New Jersey,61,51,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4a920bfdf964a520b21c20e3,40.47649684,-74.26933765,South Amboy,NJ,332,121,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4b992e4bf964a520826735e3,40.478508,-74.2819208,South Amboy,NJ,59,25,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4bc4598174a9a59339a7d5f6,40.53311118,-74.35941696,Metuchen,NJ,17,2,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4bc7afba92b376b0da43503a,40.4732721,-74.2850045,South Amboy,NJ,14,3,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4b175988f964a520f1c423e3,28.35224278,-81.60374165,Lake Buena Vista,FL,6186,2265,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4a971517f964a520562820e3,40.4778788,-74.3133737,Sayreville,NJ,39,23,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4bbb68fb7421a59366cbc440,40.437947,-74.225207,Aberdeen,New Jersey,131,56,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4be05617652b0f47f40d7211,40.4842942,-74.2808086,South Amboy,NJ,115,60,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
print('NYC的POI数目：------%d------' % len(POI_NYC))

NYC的POI数目：------206416------


In [13]:
file1 = r'E:\data\Foursquare\Microsoft_check-in data\Venues\NYC\NYC-Venues.txt'
file2 = r'D:\software\Github\Recommender-System\RS-LBSN\dataset\Foursquare\NYC.txt'
f1 = open(file1,'r',errors = 'ignore')
f2 = open(file2,'w')
for line in f1.readlines():
    cat = []
    lst = line.strip().split('\t')
    if len(lst) < 14:
        f2.write('\t'.join(lst))
        f2.write('\n')
    else:
        for e in lst[13:len(lst)]:
            if e != np.nan:
                cat.append(e[1:len(e)-1])
        lst.insert(13,','.join(cat))
        f2.write('\t'.join(lst[0:14]))
        f2.write('\n')
f1.close()
f2.close()

# 4. Check-in 分析

## LA Tips

In [11]:
###
#第一步
###
#签到数据规整
file1 = 'E:\\data\Foursquare\\Microsoft_check-in data\\Tips\\LA\\LA-Tips.txt'#最原始数据
file2 = r'E:\data\Foursquare\Microsoft_check-in data\update\check-ins\LA.txt'#第一次转换生成的，已删除
f1 = open(file1, 'r', errors = 'ignore')
f2 = open(file2, 'w')
lines = f1.readlines()
for line in lines:
    lst = line.strip().split('\t')
    con = [i for i,x in enumerate(lst) if x =='null']
    for i, j in enumerate(con):
        if i == len(con) - 1:
            p = lst[j-1:]
            p.insert(0,lst[0])
            f2.write('\t'.join(p))
            f2.write('\n')
        else:
            q = lst[j-1: con[i+1]-1] 
            q.insert(0,lst[0])
            f2.write('\t'.join(q))
            f2.write('\n')
f1.close()
f2.close()

In [14]:
###
#第二步
###
#处理种类，整合签到中的种类
file1 = r'E:\data\Foursquare\Microsoft_check-in data\update\check-ins\LA.txt' #第一步生成的数据集，已删除
file2 = 'E:\\data\\Foursquare\\Microsoft_check-in data\\update\\check-ins\\tips_LA.txt' #第二步生成的数据
f1 = open(file1,'r',errors = 'ignore')
f2 = open(file2,'w')
for line in f1.readlines():
    cat = []
    lst = line.strip().split('\t')
    if len(lst) < 9:
        f2.write('\t'.join(lst))
        f2.write('\n')
    else:
        for e in lst[8:len(lst)]:
            if e != np.nan:
                cat.append(e[1:len(e)-1])
        lst.insert(8,','.join(cat))
        f2.write('\t'.join(lst[0:9]))
        f2.write('\n')
f1.close()
f2.close()

In [28]:
Tips_LA = pd.read_table(file_LACheck, sep = '\t',encoding = 'latin-1',names = [])

D:\Python\Install\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## NYC Tips

In [15]:
#第一步，同上
#NYC签到数据规整
file1 = 'E:\\data\\Foursquare\\Microsoft_check-in data\\Tips\\NYC\\NYC-Tips.txt'
file2 = 'E:\\data\\Foursquare\\Microsoft_check-in data\\update\\check-ins\\NYC.txt'
f1 = open(file1, 'r', errors = 'ignore')
f2 = open(file2, 'w')
lines = f1.readlines()
for line in lines:
    lst = line.strip().split('\t')
    con = [i for i,x in enumerate(lst) if x =='null']
    for i, j in enumerate(con):
        if i == len(con) - 1:
            p = lst[j-1:]
            p.insert(0,lst[0])
            f2.write('\t'.join(p))
            f2.write('\n')
        else:
            q = lst[j-1: con[i+1]-1] 
            q.insert(0,lst[0])
            f2.write('\t'.join(q))
            f2.write('\n')
f1.close()
f2.close()

In [16]:
###
#第二步
###
#处理种类，整合签到中的种类
file1 = 'E:\\data\\Foursquare\\Microsoft_check-in data\\update\\check-ins\\NYC.txt' #第一步生成的数据集，已删除
file2 = 'E:\\data\\Foursquare\\Microsoft_check-in data\\update\\check-ins\\tips_NYC.txt' #第二步生成的数据
f1 = open(file1,'r',errors = 'ignore')
f2 = open(file2,'w')
for line in f1.readlines():
    cat = []
    lst = line.strip().split('\t')
    if lst[3] == '0':
        del lst[3]
    if len(lst) < 9:
        f2.write('\t'.join(lst))
        f2.write('\n')
    else:
        for e in lst[8:len(lst)]:
            if e != np.nan:
                cat.append(e[1:len(e)-1])
        lst.insert(8,','.join(cat))
        f2.write('\t'.join(lst[0:9]))
        f2.write('\n')
f1.close()
f2.close()

In [24]:
Tips_NYC = pd.read_table(file_NYCCheck, sep = '\t',encoding = 'latin-1',names = [])

D:\Python\Install\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 签到数据时间转换

In [ ]:
file_LA = r'D:\software\Github\Recommender-System\RS-LBSN\dataset\Foursquare\tips_LA.csv'
file_NYC =r'D:\software\Github\Recommender-System\RS-LBSN\dataset\Foursquare\tips_NYC.csv'
